# Final Project Task 3 - Census Modelling Regression

#### Student: Iacob Mihai

Requirements
- Create a regression model on the Census dataset, with 'hours-per-week' target

- You can use models (estmators) from sklearn, but feel free to use any library for traditional ML. 
    - Note: in sklearn, the LinearRegression estimator is based on OLS, a statistical method. Please use the SGDRegressor estimator, since this is based on gradient descent. 
    - You can use LinearRegression estimator, but only as comparison with the SGDRegressor - Optional.

- Model Selection and Setup **2p**:
    - Implement multiple models, to solve a regression problem using traditional ML: 
        - Linear Regression
        - Decision Tree Regression
        - Random Forest Regression - Optional
        - Ridge Regression - Optional
        - Lasso Regression - Optional
    - Choose a loss (or experiment with different losses) for the model and justify the choice. *1p*
        - MSE, MAE, RMSE, Huber Loss or others
    - Justify model choices based on dataset characteristics and task requirements; specify model pros and cons. *1p*


- Data Preparation
    - Use the preprocessed datasets from Task 1.
    - From the train set, create an extra validation set, if necesarry. So in total there will be: train, validation and test datasets.
    - Be sure all models have their data preprocessed as needed. Some models require different, or no encoding for some features.


- Model Training and Experimentation **10p**
    - Establish a Baseline Model *2p*
        - For each model type, train a simple model with default settings as a baseline.
        - Evaluate its performance to establish a benchmark for comparison.
    - Make plots with train, validation loss and metric on epochs (or on steps), if applicable. - Optional
    - Feature Selection: - Optional
        - Use insights from EDA in Task 2 to identify candidate features by analyzing patterns, relationships, and distributions.
    - Experimentation: *8p*
        - For each baseline model type, iteratively experiment with different combinations of features and transformations.
        - Experiment with feature engineering techniques such as interaction terms, polynomial features, or scaling transformations.
        - Identify the best model which have the best performance metrics on test set.
        - You may need multiple preprocessed datasets preprocessed
- Hyperparameter Tuning - Optional
  - Perform hyperparameter tuning only on the best-performing model after evaluating all model types and experiments. 
  - Consider using techniques like Grid Search for exhaustive tuning, Random Search for quicker exploration, or Bayesian Optimization for an intelligent, efficient search of hyperparameters.
  - Avoid tuning models that do not show strong baseline performance or are unlikely to outperform others based on experimentation.
  - Ensure that hyperparameter tuning is done after completing feature selection, baseline modeling, and experimentation, ensuring that the model is stable and representative of the dataset.


- Model Evaluation **3p**
    - Evaluate models on the test dataset using regression metrics: *1p*
        - Mean Absolute Error (MAE)
        - Mean Squared Error (MSE)
        - Root Mean Squared Error (RMSE)
        - R² Score
    - Choose one metric for model comparison and explain your choice *1p*
    - Compare the results across different models. Save all experiment results  into a table. *1p*

Feature Importance - Optional
- For applicable models (e.g., Decision Tree Regression), analyze feature importance and discuss its relevance to the problem.



Deliverables

- Notebook code with no errors.
- Code and results from experiments. Create a table with all experiments results, include experiment name, metrics results.
- Explain findings, choices, results.
- Potential areas for improvement or further exploration.


##### 1. Sanity checks and train validation set

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Loading the preprocessed datasets

X_train = pd.read_csv("D:/Facultate/ADC/Machine Learning/X_train.csv")
X_test  = pd.read_csv("D:/Facultate/ADC/Machine Learning/X_test.csv")
y_train = pd.read_csv("D:/Facultate/ADC/Machine Learning/y_train.csv")
y_test  = pd.read_csv("D:/Facultate/ADC/Machine Learning/y_test.csv")

TARGET = "hours-per-week"

print("Shapes:")
print("X_train:", X_train.shape, " y_train:", y_train.shape)
print("X_test :", X_test.shape,  " y_test :", y_test.shape)


Shapes:
X_train: (26029, 116)  y_train: (26029, 1)
X_test : (6508, 116)  y_test : (6508, 1)


In [5]:
# Checking for column consistency

assert list(X_train.columns) == list(X_test.columns), "X_train and X_test have different columns!"
print("Feature columns match.")

Feature columns match.


In [6]:
# Checking for target sanity

assert TARGET in y_train.columns and TARGET in y_test.columns, "Target column missing in y files"
y_train[TARGET] = pd.to_numeric(y_train[TARGET], errors="coerce")
y_test[TARGET]  = pd.to_numeric(y_test[TARGET], errors="coerce")

print("\nTarget quick stats (train):")
print(y_train[TARGET].describe())


Target quick stats (train):
count    26029.000000
mean        40.393868
std         12.341250
min          1.000000
25%         40.000000
50%         40.000000
75%         45.000000
max         99.000000
Name: hours-per-week, dtype: float64


In [7]:
# Missing value check

missing_X_train = X_train.isna().sum().sum()
missing_y_train = y_train.isna().sum().sum()
missing_X_test  = X_test.isna().sum().sum()
missing_y_test  = y_test.isna().sum().sum()

print("\nMissing values:")
print("X_train:", missing_X_train, "| y_train:", missing_y_train)
print("X_test :", missing_X_test,  "| y_test :", missing_y_test)



Missing values:
X_train: 0 | y_train: 0
X_test : 0 | y_test : 0


In [ ]:
# Splitting the validation set from the training set

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train[TARGET],
    test_size=0.20,
    random_state=42
)

print("\nSplit sizes:")
print("Train:", X_tr.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Split sizes:
Train: (20823, 116) Val: (5206, 116) Test: (6508, 116)


##### 2. Baseline Regression Modelling

In [9]:
# Defining metrics for model performance

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, r2

results = []

In [19]:
# SDGRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

sgd_baseline = Pipeline( [
    ("scaler", StandardScaler()),
    ("model", SGDRegressor(random_state=42))
])

# train
sgd_baseline.fit(X_tr, y_tr)

# predictions
pred_tr = sgd_baseline.predict(X_tr)
pred_val = sgd_baseline.predict(X_val)
pred_te = sgd_baseline.predict(X_test)

# metrics
mae_tr, mse_tr, rmse_tr, r2_tr = regression_metrics(y_tr, pred_tr)
mae_v, mse_v, rmse_v, r2_v = regression_metrics(y_val, pred_val)
mae_te, mse_te, rmse_te, r2_te = regression_metrics(y_test[TARGET], pred_te)

results.append({
    "model": "SGDRegressor_baseline",
    "train_MAE": mae_tr, "val_MAE": mae_v, "test_MAE": mae_te,
    "train_RMSE": rmse_tr, "val_RMSE": rmse_v, "test_RMSE": rmse_te,
    "train_R2": r2_tr, "val_R2": r2_v, "test_R2": r2_te
})


In [20]:
# Linear regression with OLS

from sklearn.linear_model import LinearRegression

lin_baseline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

lin_baseline.fit(X_tr, y_tr)

pred_tr = lin_baseline.predict(X_tr)
pred_val = lin_baseline.predict(X_val)
pred_te = lin_baseline.predict(X_test)

mae_tr, mse_tr, rmse_tr, r2_tr = regression_metrics(y_tr, pred_tr)
mae_v, mse_v, rmse_v, r2_v = regression_metrics(y_val, pred_val)
mae_te, mse_te, rmse_te, r2_te = regression_metrics(y_test[TARGET], pred_te)

results.append({
    "model": "LinearRegression_baseline",
    "train_MAE": mae_tr, "val_MAE": mae_v, "test_MAE": mae_te,
    "train_RMSE": rmse_tr, "val_RMSE": rmse_v, "test_RMSE": rmse_te,
    "train_R2": r2_tr, "val_R2": r2_v, "test_R2": r2_te
})


In [21]:
# Decision Trees

from sklearn.tree import DecisionTreeRegressor

dt_baseline = DecisionTreeRegressor(random_state=42)

dt_baseline.fit(X_tr, y_tr)

pred_tr = dt_baseline.predict(X_tr)
pred_val = dt_baseline.predict(X_val)
pred_te = dt_baseline.predict(X_test)

mae_tr, mse_tr, rmse_tr, r2_tr = regression_metrics(y_tr, pred_tr)
mae_v, mse_v, rmse_v, r2_v = regression_metrics(y_val, pred_val)
mae_te, mse_te, rmse_te, r2_te = regression_metrics(y_test[TARGET], pred_te)

results.append({
    "model": "DecisionTree_baseline",
    "train_MAE": mae_tr, "val_MAE": mae_v, "test_MAE": mae_te,
    "train_RMSE": rmse_tr, "val_RMSE": rmse_v, "test_RMSE": rmse_te,
    "train_R2": r2_tr, "val_R2": r2_v, "test_R2": r2_te
})


In [22]:
# Baseline models results

results_df = pd.DataFrame(results)
results_df


,model,train_MAE,val_MAE,test_MAE,train_RMSE,val_RMSE,test_RMSE,train_R2,val_R2,test_R2
0,SGDRegressor_baseline,742113.498240,1.023573e+06,916955.894320,3.969109e+07,4.976201e+07,4.480700e+07,-1.030760e+13,-1.649148e+13,-1.312550e+13
1,LinearRegression_baseline,7.594150,7.611115e+00,7.641111,1.083042e+01,1.083733e+01,1.088800e+01,2.325294e-01,2.178169e-01,2.249675e-01
2,DecisionTree_baseline,0.028974,1.003169e+01,10.316201,6.254487e-01,1.485355e+01,1.527505e+01,9.974405e-01,-4.693464e-01,-5.254201e-01


Linear Regression oferă performanțe stabile pe toate seturile de date, cu erori similare pe train, validation și test, indicând o relație liniară moderată între variabilele explicative și hours-per-week. 

DecisionTreeRegressor obține performanță foarte bună pe setul de antrenare, dar generalizează slab, cu erori ridicate și valori R² negative pe validation și test, semnalând overfitting semnificativ. 

În cazul SGDRegressor, configurația default produce rezultate instabile, cu erori foarte mari și R² extrem și negativ, ceea ce indică divergența optimizării. Astfel, devine necesară ajustarea funcției de pierdere și a hiperparametrilor.

Am evaluat modelele folosind MAE, MSE, RMSE și R², pentru a surprinde atât erorile medii absolute, cât și penalizarea erorilor mari și capacitatea explicativă a modelelor. Pentru comparația modelelor am folosit MAE, deoarece este mai robustă la valori extreme ale variabilei hours-per-week, care prezintă outlieri și heaping. Spre deosebire de MSE sau RMSE, MAE penalizează liniar erorile. Folosesc Linear Regression ca benchmark , deoarece oferă performanțe stabile pe toate seturile de date și surprinde relațiile liniare de bază dintre variabilele explicative și hours-per-week. Toate modelele ulterioare sunt evaluate prin comparație cu acest benchmark.

##### 3. Experimentation and Model Evaluation

In [23]:
# Experimentation setup

def metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, r2


exp_results = []

def run_and_log(name, model, Xtr, Xval, Xte, ytr, yval, yte):
    model.fit(Xtr, ytr)

    pred_tr  = model.predict(Xtr)
    pred_val = model.predict(Xval)
    pred_te  = model.predict(Xte)

    mae_tr, mse_tr, rmse_tr, r2_tr = metrics(ytr, pred_tr)
    mae_v,  mse_v,  rmse_v,  r2_v  = metrics(yval, pred_val)
    mae_te, mse_te, rmse_te, r2_te = metrics(yte, pred_te)

    exp_results.append({
        "experiment": name,
        "train_MAE": mae_tr, "val_MAE": mae_v, "test_MAE": mae_te,
        "train_MSE": mse_tr, "val_MSE": mse_v, "test_MSE": mse_te,
        "train_RMSE": rmse_tr, "val_RMSE": rmse_v, "test_RMSE": rmse_te,
        "train_R2": r2_tr, "val_R2": r2_v, "test_R2": r2_te
    })


In [24]:
TARGET = "hours-per-week"
yte = y_test[TARGET]  
print("yte min/max:", float(yte.min()), float(yte.max()))


yte min/max: 1.0 99.0


Am implementat un cadru standard de evaluare care calculează MAE, MSE, RMSE și R² pe seturile de train, validation și test și care stochează rezultatele tuturor experimentelor într-un tabel comparabil.

In [ ]:
# Feature sets (all + EDA)

# All
Xtr_all, Xval_all, Xte_all = X_tr.copy(), X_val.copy(), X_test.copy()

# EDA-based subset
eda_cols = [c for c in X_tr.columns if (
    ("education" in c) or
    ("occupation" in c) or
    ("workclass" in c) or
    ("marital" in c) or
    ("relationship" in c) or
    ("race" in c) or
    ("sex" in c) or
    (c == "age") or
    ("capital-gain" in c) or
    ("capital-loss" in c)
)]

print("ALL features:", X_tr.shape[1])
print("EDA subset features:", len(eda_cols))

# subset dataframes
Xtr_eda = X_tr[eda_cols].copy()
Xval_eda = X_val[eda_cols].copy()
Xte_eda  = X_test[eda_cols].copy()

# target for test
yte = y_test[TARGET]

Xtr_eda.shape, Xval_eda.shape, Xte_eda.shape



ALL features: 116
EDA subset features: 68


((20823, 68), (5206, 68), (6508, 68))

Am definit două seturi de feature-uri pentru experimentare: ALL features (toate variabilele preprocesate) și un subset EDA-informed (educație, ocupație, workclass, variabile socio-demografice și capital gain/loss). Scopul este să evaluăm dacă reducerea dimensionalității și concentrarea pe variabile relevante îmbunătățește generalizarea pe setul de test.

In [ ]:
# Using Log for highly skewed variables

def add_log1p_capitals(X):
    X2 = X.copy()
    for col in ["capital-gain", "capital-loss"]:
        if col in X2.columns:
            X2[f"log1p_{col}"] = np.log1p(
                pd.to_numeric(X2[col], errors="coerce").fillna(0)
            )
    return X2

# log-transformed datasets

Xtr_log = add_log1p_capitals(Xtr_all)
Xval_log = add_log1p_capitals(Xval_all)
Xte_log  = add_log1p_capitals(Xte_all)

Xtr_log.shape, Xtr_all.shape

((20823, 118), (20823, 116))

Am folosit transformări logaritmice pentru variabile extrem de skewed cu scopul de a reduce influența valorilor extreme și de a îmbunătăți stabilitatea modelelor liniare.

In [28]:
# Polynomial features

from sklearn.preprocessing import PolynomialFeatures

poly_base = [c for c in ["age", "capital-gain", "capital-loss"] if c in Xtr_all.columns]
print("Polynomial base variables:", poly_base)

def make_poly_dataset(X, base_cols, degree=2):
    if len(base_cols) == 0:
        return X.copy()

    rest_cols = [c for c in X.columns if c not in base_cols]
    X_base = X[base_cols]

    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X_base)

    poly_names = poly.get_feature_names_out(base_cols)
    X_poly_df = pd.DataFrame(X_poly, columns=poly_names, index=X.index)

    return pd.concat([X_poly_df, X[rest_cols]], axis=1)

# Polynomes and interactions datasets 

Xtr_poly = make_poly_dataset(Xtr_all, poly_base, degree=2)
Xval_poly = make_poly_dataset(Xval_all, poly_base, degree=2)
Xte_poly  = make_poly_dataset(Xte_all,  poly_base, degree=2)

Xtr_poly.shape, Xtr_all.shape


Polynomial base variables: ['age', 'capital-gain', 'capital-loss']


((20823, 122), (20823, 116))

Am testat features polinomiale și interacțiuni de ordinul doi pentru un subset restrâns de variabile numerice (age, capital-gain, capital-loss), pentru a surprinde posibile relații neliniare. Transformarea a fost aplicată controlat, pentru a evita creșterea excesivă a dimensionalității.

In [29]:
# Linear Regression (benchmark, all + EDA)

# ALL features

lin_all = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])
run_and_log("LinearRegression_scaled_ALL",
            lin_all, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)

# EDA subset

lin_eda = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])
run_and_log("LinearRegression_scaled_EDA",
            lin_eda, Xtr_eda, Xval_eda, Xte_eda, y_tr, y_val, yte)


Folosesc regresia liniară ca benchmark. Antrenarea pe un subset EDA-informed permite evaluarea impactului reducerii dimensionalității asupra performanței.

In [30]:
# Ridge (L2) Stable for Colinearity

from sklearn.linear_model import Ridge

ridge_a1 = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=1.0))
])
run_and_log("Ridge_alpha1_ALL",
            ridge_a1, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)

ridge_a10 = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=10.0))
])
run_and_log("Ridge_alpha10_ALL",
            ridge_a10, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)


In [31]:
# Lasso (L1) Implicit Feature Selection

from sklearn.linear_model import Lasso

lasso_1 = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Lasso(alpha=0.001, max_iter=5000))
])
run_and_log("Lasso_alpha0.001_ALL",
            lasso_1, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)


Ridge și Lasso au fost folosite pentru a evalua efectul regularizării asupra modelelor liniare, în special în contextul numărului mare de variabile codate cu one-hot-encoder.

In [32]:
#SGDRegressor with Huber loss - outlier resistant

sgd_huber_all = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SGDRegressor(
        loss="huber",
        epsilon=1.35,
        alpha=1e-4,
        learning_rate="invscaling",
        eta0=0.01,
        max_iter=5000,
        tol=1e-3,
        random_state=42
    ))
])
run_and_log("SGD_Huber_ALL",
            sgd_huber_all, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)


In [33]:
#SGDRegressor Huber on EDA Subset

sgd_huber_eda = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SGDRegressor(
        loss="huber",
        epsilon=1.35,
        alpha=1e-4,
        learning_rate="invscaling",
        eta0=0.01,
        max_iter=5000,
        tol=1e-3,
        random_state=42
    ))
])
run_and_log("SGD_Huber_EDA",
            sgd_huber_eda, Xtr_eda, Xval_eda, Xte_eda, y_tr, y_val, yte)


In [34]:
# SGDRegressor Huber with polynomial features

sgd_huber_poly = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SGDRegressor(
        loss="huber",
        epsilon=1.35,
        alpha=1e-4,
        learning_rate="invscaling",
        eta0=0.01,
        max_iter=5000,
        tol=1e-3,
        random_state=42
    ))
])
run_and_log("SGD_Huber_Polynomial",
            sgd_huber_poly, Xtr_poly, Xval_poly, Xte_poly, y_tr, y_val, yte)


In [35]:
# Decision Tree baseline vs regularizat

from sklearn.tree import DecisionTreeRegressor

dt_base = DecisionTreeRegressor(random_state=42)
run_and_log("DecisionTree_baseline",
            dt_base, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)

dt_reg = DecisionTreeRegressor(
    random_state=42,
    max_depth=12,
    min_samples_leaf=20
)
run_and_log("DecisionTree_regularized",
            dt_reg, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)


In [36]:
# Random Forest

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=400,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)
run_and_log("RandomForest_400_leaf5",
            rf, Xtr_all, Xval_all, Xte_all, y_tr, y_val, yte)


Am folosit modele de tip decision tree si random forest deoarece permit captarea relațiilor non-liniare și a interacțiunilor dintre variabile. Mai mult, random forest reduce overfitting-ul prin agregarea mai multor arbori.

In [37]:
# Final table

exp_df = pd.DataFrame(exp_results)

exp_df_sorted = exp_df.sort_values("test_MAE")

exp_df_sorted[[
    "experiment",
    "val_MAE", "test_MAE",
    "val_RMSE", "test_RMSE",
    "val_R2", "test_R2"
]].head(10)


,experiment,val_MAE,test_MAE,val_RMSE,test_RMSE,val_R2,test_R2
7,SGD_Huber_Polynomial,7.238068,7.265582,11.035579,11.119830,0.188938,0.191611
5,SGD_Huber_ALL,7.296948,7.337199,11.197766,11.262833,0.164923,0.170686
10,RandomForest_400_leaf5,7.296031,7.338819,10.617388,10.685355,0.249243,0.253548
6,SGD_Huber_EDA,7.355714,7.426453,11.516921,11.605958,0.116643,0.119385
9,DecisionTree_regularized,7.450267,7.498203,10.846666,10.950247,0.216469,0.216080
4,Lasso_alpha0.001_ALL,7.610670,7.640544,10.837263,10.887714,0.217827,0.225008
3,Ridge_alpha10_ALL,7.610909,7.640901,10.837326,10.887945,0.217818,0.224975
2,Ridge_alpha1_ALL,7.611095,7.641090,10.837331,10.887991,0.217817,0.224968
0,LinearRegression_scaled_ALL,7.611115,7.641111,10.837331,10.887996,0.217817,0.224967
1,LinearRegression_scaled_EDA,7.760068,7.796661,11.156257,11.218165,0.171103,0.177251


In [38]:
# Best model from the experiments

best = exp_df_sorted.iloc[0]
print("Best model:", best["experiment"])
print("Test MAE:", best["test_MAE"], " | Test R2:", best["test_R2"])


Best model: SGD_Huber_Polynomial
Test MAE: 7.265582156228628  | Test R2: 0.19161115773600423


Compararea experimentelor pe baza MAE pe setul de test a indicat faptul că modelul SGDRegressor cu Huber loss și polynomial features de ordinul doi oferă cea mai bună performanță predictivă, cu o MAE de 7.27 ore și un R² de 0.19. Rezultatul sugerează că o optimizare robustă la outlieri, combinată cu introducerea controlată a non-liniarităților, îmbunătățește capacitatea de generalizare față de modelele liniare standard și față de cele non-liniare.

##### 4. Limitations and future work

Din punct de vedere al modelării, performanța predictivă este constrânsă de caracterul discret, zero-inflated și heteroscedastic al variabilei hours-per-week, care reduce eficiența metodelor de regresie standard optimizate pentru erori continue. Deși modelele bazate pe gradient descent și transformări nonn-liniare surprind tipare locale, o parte substanțială a variației rămâne neexplicată din cauza factorilor instituționali și organizaționali neobservați, precum regimul contractual, normele ocupaționale sau practicile informale de muncă. Direcții viitoare pot include optimizarea hiperparametrică focalizată, utilizarea unor loss functions adaptate distribuțiilor discrete și explorarea unor formulări alternative ale problemei (de exemplu, regresie ordinală sau modele ierarhice), care ar permite integrarea mai explicită a constrângerilor sociale ce structurează timpul de muncă.